In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs4
import plotly.graph_objects as go
import json

In [2]:
driver = webdriver.Firefox(executable_path="/Users/vojtechjindra/Downloads/geckodriver")

In [15]:
creds = pd.read_json('credentials.json', orient='index')
login, password = creds.loc['login', 0], creds.loc['password', 0]

In [16]:
driver.get("https://app.click4survey.cz/Login.aspx")

driver.find_element_by_id("Email").send_keys(login)
driver.find_element_by_id("Password").send_keys(password)
driver.find_element_by_class_name("button-green").click()

In [17]:
driver.find_element_by_class_name("reports").click()

In [377]:
def get_next_page_button():
    pages = driver.find_elements_by_class_name("pager")[0].find_elements_by_tag_name('a')
    return list(filter(lambda p: p.get_attribute('innerHTML') == '»', pages))

In [421]:
cols = [
    'ID', 'Gender', 'Age', 'BMI', 'Email', 
    'Question', 'Nikdy', 'Méně než měsíčně', 
    '1x měsíčně', '2 - 3x měsíčně', '1x týdně',
    '2 - 3x týdně', '4 - 6x týdně', 'Každý den'
]

df = pd.DataFrame(columns=cols)

run = True
while run:
    table = bs4(driver.find_element_by_class_name("list").get_attribute("innerHTML"))
    
    els = [e.find('a') for e in table.findAll('td')]
    els = [e for e in els if e is not None]
    els = [e.get('data-ldata') for e in els]
        
    for el in els:
        driver.find_element(By.XPATH, f"//a[@data-ldata='{el}']").click()

        answers = driver.find_elements_by_class_name("question-answer-value")
        while len(answers) == 0:
            answers = driver.find_elements_by_class_name("question-answer-value")

        gender = bs4(answers[0].get_attribute('innerHTML')).text
        age = bs4(answers[1].get_attribute('innerHTML')).text
        bmi = bs4(answers[2].get_attribute('innerHTML')).text
        email = bs4(answers[3].get_attribute('innerHTML')).text if len(answers) > 3 else ''

        q = bs4(driver.find_elements_by_class_name("question")[-3].get_attribute("innerHTML"))

        rows = q.find_all("tr")
        rows = rows[1:]
        for row in rows:
            cells = row.find_all('td')
            question = cells[0].select('span')[0].text
            cells = cells[1:]

            res = [el, gender, age, bmi, email, question] + [1 if len(cell.decode_contents()) > 0 else 0 for cell in cells]
            df = df.append({cols[i]: res[i] for i in range(len(cols))}, ignore_index=True)

        while len(driver.find_elements_by_class_name('c4s-lightbox-overlay')) > 0:
            close_button = driver.find_elements_by_class_name("close-button")

            if len(close_button) > 0:
                close_button[0].click()
    
    next_page = get_next_page_button()
    
    if len(next_page) > 0:
        next_page[0].click()
        sleep(1)
    else:
        run = False

In [422]:
def get_threshold(q):
    if q in [3, 7, 12, 13]:
        return 2
    if q in [1, 4, 8, 10]:
        return 4
    if q in [2, 5, 6, 9, 11]:
        return 5

def get_val(x):
    cols = [
        'Nikdy', 'Méně než měsíčně', '1x měsíčně',
        '2 - 3x měsíčně', '1x týdně', '2 - 3x týdně',
        '4 - 6x týdně', 'Každý den'
    ]
    
    for i in range(len(cols)):
        if x[cols[i]] == 1:
            return i
        
    return -1

def calc_score(x):
    res = 0
    double_q = False
    for q in range(13):
        threshold = get_threshold(q+1)
        val = get_val(x.iloc[q])
        
        if val >= threshold:
            if q == 5 or q == 6:
                if not double_q:
                    res += 1
                double_q = True
            else:
                res += 1
    
    return res

In [423]:
scores = df.groupby('ID').apply(calc_score)

res = df.groupby('ID').agg({
    'Gender': 'first',
    'Age': 'first',
    'Email': 'first',
    'BMI': 'first'
})

res['Score'] = scores
res['BMI'] = res['BMI'].str.strip()

In [440]:
res[res['BMI'] == 'Ano']

,Gender,Age,Email,BMI,Score
ID,,,,,
5798162,Muž,18 - 30 let,ukaž kozy,Ano,2
5810885,Muž,18 - 30 let,domip@seznam.cz,Ano,5
5810925,Muž,18 - 30 let,Bělča 777101370,Ano,1
5857287,Muž,18 - 30 let,taddy.fort@gmail.com,Ano,0
5858722,Žena,18 - 30 let,"Nechci, omlouvám se",Ano,2
5859668,Žena,18 - 30 let,,Ano,9
5875059,Žena,18 - 30 let,,Ano,0
5899912,Žena,31 - 50 let,zavadra@seznam.cz,Ano,0
5994150,Žena,31 - 50 let,jitkamocova@centrum.cz,Ano,0
